# IG to Test Kit FULL pipeline

## Setup

### Importing Notebooks as Modules (from the [Jupyter Notebook Documentation](https://jupyter-notebook.readthedocs.io/en/4.x/examples/Notebook/rstversions/Importing%20Notebooks.html))

In [9]:
import inspect
import json
import llm_utils
import importlib
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from glob import glob

## Initializing LLM Clients

In [17]:
importlib.reload(llm_utils)
llm_clients = llm_utils.LLMApiClient()

In [14]:
llm_clients.clients

{'claude': <anthropic.Anthropic at 0x12fb732c0>,
 'gemini': genai.GenerativeModel(
     model_name='models/gemini-2.5-pro-preview-03-25',
     generation_config={'max_output_tokens': 8192, 'temperature': 0.3},
     safety_settings={},
     tools=None,
     system_instruction=None,
     cached_content=None
 ),
 'gpt': <openai.OpenAI at 0x12beeb470>}

## Text Extraction

### HTML to Markdown Conversion Using Markdownify (Langchain Tool)

In [5]:
def is_absolute(url):
    return bool(urlparse(url).netloc)

base = 'https://hl7.org/fhir/us/davinci-pdex-plan-net'
soup = BeautifulSoup(requests.get(base+'/artifacts.html').text)

l_list = [x.get('href', '') for x in soup.find_all('a')]
l_start = l_list.index("CapabilityStatement-plan-net.html")
l_stop = l_start+l_list[l_start:].index('security.html')
l_list = l_list[46:l_stop]
l2 = []
for l in l_list:
    if l not in ['#', '']:
        if is_absolute(l):
            l2.append(l)
        else:
            l2.append(base+'/'+l)

/var/folders/y4/5w3wfdld3gv5brcy7cz9m7f80000gn/T/ipykernel_5933/2520396402.py:5: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(requests.get(base+'/artifacts.html').text)


In [8]:
for l in list(set(l2)):
    with open(f"checkpoints/html_download/{l.split('/')[-1]}", 'w+') as f:
        f.write(requests.get(l).text)

In [13]:
import HTML_extractor 
importlib.reload(HTML_extractor)

<module 'HTML_extractor' from '/Users/jrockhill/ONCLAIVE/pipeline/HTML_extractor.py'>

In [ ]:
HTML_extractor.convert_local_html_to_markdown(input_dir='checkpoints/html_download/', output_dir="checkpoints/text_extraction/PlanNet/with_sections/markdown")

Found 165 HTML files to process


/Users/jrockhill/ONCLAIVE/pipeline/HTML_extractor.py:76: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 76 of the file /Users/jrockhill/ONCLAIVE/pipeline/HTML_extractor.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html_content)
/Users/jrockhill/ONCLAIVE/pipeline/HTML_extractor.py:76: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml packa

Processed 10/165 files
Processed 20/165 files
Processed 30/165 files
Processed 40/165 files
Processed 50/165 files
Processed 60/165 files
Processed 70/165 files
Processed 80/165 files
Processed 90/165 files
Processed 100/165 files
Processed 110/165 files
Processed 120/165 files
Processed 130/165 files
Processed 140/165 files
Processed 150/165 files
Processed 160/165 files
Processed 165/165 files
Conversion complete. Successfully processed 165 files. Encountered 0 errors.


### Markdown Post-processing

In [15]:
import markdown_cleaner
markdown_cleaner.process_directory("checkpoints/text_extraction/PlanNet/with_sections/markdown", "checkpoints/post_processing/PlanNet/")

Found 165 markdown files in checkpoints/text_extraction/PlanNet/with_sections/markdown
Cleaned and saved: checkpoints/post_processing/PlanNet/Location-HospLoc1.md
Cleaned and saved: checkpoints/post_processing/PlanNet/SearchParameter-location-endpoint.md
Cleaned and saved: checkpoints/post_processing/PlanNet/HealthcareService-HartfordOrthopedicServices.md
Cleaned and saved: checkpoints/post_processing/PlanNet/PractitionerRole-JoeSmithRole2.md
Cleaned and saved: checkpoints/post_processing/PlanNet/SearchParameter-healthcareservice-location.md
Cleaned and saved: checkpoints/post_processing/PlanNet/SearchParameter-practitionerrole-period.md
Cleaned and saved: checkpoints/post_processing/PlanNet/SearchParameter-healthcareservice-endpoint.md
Cleaned and saved: checkpoints/post_processing/PlanNet/CodeSystem-DeliveryMethodCS.md
Cleaned and saved: checkpoints/post_processing/PlanNet/HealthcareService-PharmChainRetailService.md
Cleaned and saved: checkpoints/post_processing/PlanNet/SearchParame

## Requirements Extraction

### Prompt-based Requirement Extraction

In [16]:
import reqs_extraction
importlib.reload(reqs_extraction)

INFO:root:Current working directory: /Users/jrockhill/ONCLAIVE/pipeline
INFO:root:Project root: /Users/jrockhill/ONCLAIVE
INFO:root:Prompt environment set up at: /Users/jrockhill/ONCLAIVE/prompts
INFO:root:Using prompts directory: /Users/jrockhill/ONCLAIVE/prompts
INFO:root:Requirements extraction prompt: /Users/jrockhill/ONCLAIVE/prompts/requirements_extraction.md
INFO:root:Current working directory: /Users/jrockhill/ONCLAIVE/pipeline
INFO:root:Project root: /Users/jrockhill/ONCLAIVE
INFO:root:Prompt environment set up at: /Users/jrockhill/ONCLAIVE/prompts
INFO:root:Using prompts directory: /Users/jrockhill/ONCLAIVE/prompts
INFO:root:Requirements extraction prompt: /Users/jrockhill/ONCLAIVE/prompts/requirements_extraction.md


<module 'reqs_extraction' from '/Users/jrockhill/ONCLAIVE/pipeline/reqs_extraction.py'>

In [ ]:
reqs_extraction.run_requirements_extractor(
    'checkpoints/post_processing/PlanNet', 
    'checkpoints/requirements_extraction/markdown/PlanNet', 
    'claude', 
    llm_clients)

INFO:root:Found markdown directory at checkpoints/post_processing/PlanNet
INFO:root:Found 165 markdown files
INFO:root:Processing with claude...
INFO:root:Starting processing with claude on directory: checkpoints/post_processing/PlanNet
INFO:root:Found 165 markdown files:
INFO:root:  - Location-HospLoc1.md
INFO:root:  - SearchParameter-location-endpoint.md
INFO:root:  - HealthcareService-HartfordOrthopedicServices.md
INFO:root:  - PractitionerRole-JoeSmithRole2.md
INFO:root:  - SearchParameter-healthcareservice-location.md
INFO:root:  - SearchParameter-practitionerrole-period.md
INFO:root:  - SearchParameter-healthcareservice-endpoint.md
INFO:root:  - CodeSystem-DeliveryMethodCS.md
INFO:root:  - HealthcareService-PharmChainRetailService.md
INFO:root:  - SearchParameter-insuranceplan-identifier.md
INFO:root:  - SearchParameter-healthcareservice-organization.md
INFO:root:  - OrganizationAffiliation-HamiltonClinicAffil.md
INFO:root:  - StructureDefinition-plannet-PractitionerRole.md
INFO:


Processing Implementation Guide with Claude...
This may take several minutes depending on the size of the Implementation Guide.


INFO:root:Organized 165 files into 65 processing groups
INFO:root:Processing combined group of 7 files
INFO:root:Split combined content into 2 chunks
INFO:root:Processing chunk 1/2 of combined files
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Processing chunk 2/2 of combined files
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Processing combined group of 7 files
INFO:root:Split combined content into 2 chunks
INFO:root:Processing chunk 1/2 of combined files
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Processing chunk 2/2 of combined files
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:root:Processing combined group of 6 files
INFO:root:Split combined content into 2 chunks
INFO:root:Processing chunk 1/2 of combined files
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP

### RAG-based Requirement Extraction

This extraction requirement extraction method differs from the first in that, as a part of the creation of its prompt, it performs a semantic search on example sections of FHIR IG text and the human-generated requirements that were produced in reference to those sections of text to find the most similar section(s) of FHIR IG text in the database and their associated requirement(s). Those sets of IG text and requirement(s) are then supplied to the LLM as few-shot examples

In [3]:
import rag_reqs_extraction

In [ ]:
importlib.reload(rag_reqs_extraction)
rag_reqs_extraction.full_pass(llm_clients, 'claude', "checkpoints/post_processing/", "checkpoints/requirements_extraction/RAG")

## Requirement Downselection

In [24]:
import requirement_downselect
importlib.reload(requirement_downselect)
requirement_downselect.full_pass(
    md_files=["checkpoints/requirements_extraction/claude_reqs_list_v1_20250429_081756.md"],
    rag_files=["checkpoints/requirements_extraction/RAG/plan_net_reqs.json"]
    )

## Test Plan Generation

In [31]:
import logging
llm_clients.logger.setLevel(logging.INFO)

In [10]:
import req_to_testplan
importlib.reload(req_to_testplan)

req_to_testplan.generate_consolidated_test_plan(
    llm_clients, 
    'claude', 
    llm_clients.logger, 
    "/Users/ceadams/Documents/onclaive/onclaive/reqs_extraction/revised_reqs_output/plan-net-requirements.md", 
    "../full-ig/markdown7_cleaned/CapabilityStatement_plan_net.md", 
    "Plan-Net IG",
    output_dir='checkpoints/testplan_generation')

2025-06-20 11:48:36,696 - root - INFO - Prompt environment set up at: /Users/ceadams/Documents/onclaive/onclaive/prompts
2025-06-20 11:48:36,697 - llm_utils - INFO - Starting test plan generation with claude for Plan-Net IG
2025-06-20 11:48:36,698 - llm_utils - ERROR - Error processing requirements: Expecting value: line 1 column 1 (char 0)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Test Kit Generation

In [8]:
import plan_to_tests
importlib.reload(plan_to_tests)

plan_to_tests.generate_inferno_test_kit(
    llm_clients,
    'claude',
    '/Users/ceadams/Documents/onclaive/onclaive/test_kit_dev/test_plan_output/example_claude_test_plan_20250416_143409.md',
    #'../test_kit_dev/inferno-guidance.md',
    output_dir='checkpoints/testkit_generation',
    expected_actors=["Health Plan API Actor", "Application Actor"]
)

2025-06-19 15:10:47,505 - plan_to_tests - INFO - Starting Inferno test generation with claude for PlanNet
2025-06-19 15:10:47,516 - plan_to_tests - INFO - Parsed test plan into 11 sections
2025-06-19 15:10:47,516 - plan_to_tests - INFO - Found 11 total requirements
2025-06-19 15:10:47,518 - plan_to_tests - INFO - Loaded Inferno DSL guidance
2025-06-19 15:10:47,518 - plan_to_tests - INFO - Processing section: Application-Level Requirements with 1 requirements
2025-06-19 15:10:47,518 - plan_to_tests - INFO - Generating tests for section: Application-Level Requirements
2025-06-19 15:10:47,519 - plan_to_tests - INFO - Generating test for requirement: REQ-08
2025-06-19 15:10:47,520 - plan_to_tests - INFO - Requirement REQ-08: Sending 989 tokens to claude API (limit: 16000)


Found 11 potential requirements
Processing requirement: REQ-08
Added requirement REQ-08 to section Application-Level Requirements
Processing requirement: REQ-01
Added requirement REQ-01 to section Authentication
Processing requirement: REQ-09
Added requirement REQ-09 to section Base Requirements
Processing requirement: REQ-07
Added requirement REQ-07 to section CORE Conformance
Processing requirement: REQ-06
Added requirement REQ-06 to section Cross-Resource
Processing requirement: REQ-04
Added requirement REQ-04 to section General Requirements
Processing requirement: REQ-05
Added requirement REQ-05 to section Global
Processing requirement: REQ-11
Added requirement REQ-11 to section OrganizationAffiliation
Processing requirement: REQ-02
Added requirement REQ-02 to section Plan-Net API Security
Processing requirement: REQ-10
Added requirement REQ-10 to section PractitionerRole
Processing requirement: REQ-03
Added requirement REQ-03 to section Security


2025-06-19 15:10:58,370 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-06-19 15:10:58,371 - plan_to_tests - INFO - Successfully generated test for requirement: REQ-08
2025-06-19 15:10:58,372 - plan_to_tests - INFO - Validating test for requirement: REQ-08
2025-06-19 15:10:58,372 - plan_to_tests - INFO - Validation for test: Sending 1528 tokens to claude API
2025-06-19 15:11:07,548 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-06-19 15:11:07,552 - plan_to_tests - INFO - Successfully validated test for requirement: REQ-08
2025-06-19 15:11:10,556 - plan_to_tests - INFO - Processing section: Authentication with 1 requirements
2025-06-19 15:11:10,558 - plan_to_tests - INFO - Generating tests for section: Authentication
2025-06-19 15:11:10,562 - plan_to_tests - INFO - Generating test for requirement: REQ-01
2025-06-19 15:11:10,565 - plan_to_tests - INFO - Requirement REQ-01: Sending 983 token

{'total_sections': 11,
 'total_requirements': 11,
 'generated_tests': 11,
 'module_dir': 'checkpoints/testkit_generation/plannet',
 'module_file': 'checkpoints/testkit_generation/plannet.rb'}